In [1]:
import sys
sys.path.append('C:/Work/pythonshizai/07_src')

In [2]:
import numpy as np
import pandas as pd
import argparse
import inject
import sys
import os
from configparser import ConfigParser

In [3]:
import tensorflow as tf

In [4]:
from C7013.lib import const, message
from C7013.lib.C7013_04_rank_training_task import C7013_04_rank_training_task
from C7013.lib.rank_model_helper import RankModelHelper
from C7013.lib.rank_model_helper import RankTrainingGenerator
from C7013.lib.C7013_04_rank_flag_forced_correction_task import C7013_04_rank_flag_forced_correction_task 
from C7013.lib import rank_utils
from C7013.lib import onehot_utils

In [5]:
print('ランク学習タスクを実行します。')

ランク学習タスクを実行します。


In [6]:
rank_model_helper: RankModelHelper = RankModelHelper()

In [7]:
epochs = const.APP_CONFIG['rank_config']['epochs']
batch_size = const.APP_CONFIG['rank_config']['batch_size']
workers = const.APP_CONFIG['rank_config']['workers']

In [8]:
print(f'Training Rank Model batch_size={batch_size}, epochs={epochs}, workers={workers}')

Training Rank Model batch_size=32, epochs=20, workers=1


In [9]:
training_file_path = 'C:/Work/PythonShizai/07_src/C7013/data/rank_training_data.csv'

In [10]:
validation_file_path = 'C:/Work/PythonShizai/07_src/C7013/data/rank_validation_data.csv'

In [12]:
input_data = pd.read_csv(training_file_path, sep=',', encoding='utf-8')
if input_data.empty:
    raise RuntimeError(f"データが存在しません。file_path={training_file_path}")
flag_forced_correction = C7013_04_rank_flag_forced_correction_task()
_input_data: pd.DataFrame = flag_forced_correction.execute(input_data)

[2021-03-03 15:14:20,824][INFO    ](C7013_04_rank_flag_forced_correction_task.py:37) フラグ強制補正タスクを実行します。


In [13]:
_input_data.head()

,rank_system,rule_no,ordercontents,rank_flag01,rank_flag02,rank_flag03,rank_flag04,rank_flag05,rank_flag06,rank_flag07,...,rank_flag62,rank_flag63,rank_flag64,rank_flag65,rank_flag66,rank_flag67,rank_flag68,rank_flag69,rank_flag70,rank_flag71
0,300,12,9,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,500,36,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,400,26,2,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,400,21,10,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,500,31;39,9,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
val_data = pd.read_csv(validation_file_path, sep=',', encoding='utf-8')
if val_data.empty:
    raise RuntimeError(f"データが存在しません。file_path={validation_file_path}")
flag_forced_correction = C7013_04_rank_flag_forced_correction_task()
_val_data: pd.DataFrame = flag_forced_correction.execute(val_data)

[2021-03-03 15:14:30,324][INFO    ](C7013_04_rank_flag_forced_correction_task.py:37) フラグ強制補正タスクを実行します。


In [16]:
train_examples = rank_utils.input_data_transform(_input_data)
validation_examples = rank_utils.input_data_transform(_val_data)

In [17]:
train_examples.shape

(43829, 83)

In [18]:
validation_examples.shape

(10565, 83)

In [19]:
labels_onehot = rank_utils.target_data_transform(_input_data)
val_labels_onehot = rank_utils.target_data_transform(_val_data)

In [20]:
labels_onehot.shape

(43829, 5)

In [21]:
train_labels_a = labels_onehot[:,0:1]
train_labels_b = labels_onehot[:,1:2]
train_labels_c = labels_onehot[:,2:3]
train_labels_d = labels_onehot[:,3:4]
train_labels_bar = labels_onehot[:,4:5]

In [22]:
val_labels_a = val_labels_onehot[:,0:1]
val_labels_b = val_labels_onehot[:,1:2]
val_labels_c = val_labels_onehot[:,2:3]
val_labels_d = val_labels_onehot[:,3:4]
val_labels_bar = val_labels_onehot[:,4:5]

In [25]:
train_dataset_a = tf.data.Dataset.from_tensor_slices((train_examples, train_labels_a))
train_dataset_b = tf.data.Dataset.from_tensor_slices((train_examples, train_labels_b))
train_dataset_c = tf.data.Dataset.from_tensor_slices((train_examples, train_labels_c))
train_dataset_d = tf.data.Dataset.from_tensor_slices((train_examples, train_labels_d))
train_dataset_bar = tf.data.Dataset.from_tensor_slices((train_examples, train_labels_bar))

In [26]:
validation_dataset_a = tf.data.Dataset.from_tensor_slices((validation_examples, val_labels_a))
validation_dataset_b = tf.data.Dataset.from_tensor_slices((validation_examples, val_labels_b))
validation_dataset_c = tf.data.Dataset.from_tensor_slices((validation_examples, val_labels_c))
validation_dataset_d = tf.data.Dataset.from_tensor_slices((validation_examples, val_labels_d))
validation_dataset_bar = tf.data.Dataset.from_tensor_slices((validation_examples, val_labels_bar))

In [28]:
SHUFFLE_BUFFER_SIZE = 100

In [29]:
train_dataset_a = train_dataset_a.shuffle(SHUFFLE_BUFFER_SIZE).batch(batch_size)
train_dataset_b = train_dataset_b.shuffle(SHUFFLE_BUFFER_SIZE).batch(batch_size)
train_dataset_c = train_dataset_c.shuffle(SHUFFLE_BUFFER_SIZE).batch(batch_size)
train_dataset_d = train_dataset_d.shuffle(SHUFFLE_BUFFER_SIZE).batch(batch_size)
train_dataset_bar = train_dataset_bar.shuffle(SHUFFLE_BUFFER_SIZE).batch(batch_size)

In [30]:
validation_dataset_a = validation_dataset_a.shuffle(SHUFFLE_BUFFER_SIZE).batch(batch_size)
validation_dataset_b = validation_dataset_b.shuffle(SHUFFLE_BUFFER_SIZE).batch(batch_size)
validation_dataset_c = validation_dataset_c.shuffle(SHUFFLE_BUFFER_SIZE).batch(batch_size)
validation_dataset_d = validation_dataset_d.shuffle(SHUFFLE_BUFFER_SIZE).batch(batch_size)
validation_dataset_bar = validation_dataset_bar.shuffle(SHUFFLE_BUFFER_SIZE).batch(batch_size)

In [32]:
_rank_a_model = rank_model_helper._create_binary_crossentropy_model('rank_a_model')
_rank_a_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
_rank_a_model.summary()

Model: "rank_a_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rank_input (InputLayer)      [(None, 83)]              0         
_________________________________________________________________
middle_layer1 (Dense)        (None, 55)                4620      
_________________________________________________________________
middle_layer2 (Dropout)      (None, 55)                0         
_________________________________________________________________
rank_output (Dense)          (None, 1)                 56        
Total params: 4,676
Trainable params: 4,676
Non-trainable params: 0
_________________________________________________________________


In [33]:
_rank_b_model = rank_model_helper._create_binary_crossentropy_model('rank_b_model')
_rank_b_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
_rank_b_model.summary()

Model: "rank_b_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rank_input (InputLayer)      [(None, 83)]              0         
_________________________________________________________________
middle_layer1 (Dense)        (None, 55)                4620      
_________________________________________________________________
middle_layer2 (Dropout)      (None, 55)                0         
_________________________________________________________________
rank_output (Dense)          (None, 1)                 56        
Total params: 4,676
Trainable params: 4,676
Non-trainable params: 0
_________________________________________________________________


In [34]:
_rank_c_model = rank_model_helper._create_binary_crossentropy_model('rank_c_model')
_rank_c_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
_rank_c_model.summary()

Model: "rank_c_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rank_input (InputLayer)      [(None, 83)]              0         
_________________________________________________________________
middle_layer1 (Dense)        (None, 55)                4620      
_________________________________________________________________
middle_layer2 (Dropout)      (None, 55)                0         
_________________________________________________________________
rank_output (Dense)          (None, 1)                 56        
Total params: 4,676
Trainable params: 4,676
Non-trainable params: 0
_________________________________________________________________


In [35]:
_rank_d_model = rank_model_helper._create_binary_crossentropy_model('rank_d_model')
_rank_d_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
_rank_d_model.summary()

Model: "rank_d_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rank_input (InputLayer)      [(None, 83)]              0         
_________________________________________________________________
middle_layer1 (Dense)        (None, 55)                4620      
_________________________________________________________________
middle_layer2 (Dropout)      (None, 55)                0         
_________________________________________________________________
rank_output (Dense)          (None, 1)                 56        
Total params: 4,676
Trainable params: 4,676
Non-trainable params: 0
_________________________________________________________________


In [36]:
_rank_bar_model = rank_model_helper._create_binary_crossentropy_model('rank_bar_model')
_rank_bar_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
_rank_bar_model.summary()

Model: "rank_bar_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rank_input (InputLayer)      [(None, 83)]              0         
_________________________________________________________________
middle_layer1 (Dense)        (None, 55)                4620      
_________________________________________________________________
middle_layer2 (Dropout)      (None, 55)                0         
_________________________________________________________________
rank_output (Dense)          (None, 1)                 56        
Total params: 4,676
Trainable params: 4,676
Non-trainable params: 0
_________________________________________________________________


In [37]:
history_a = _rank_a_model.fit(train_dataset_a, validation_data=validation_dataset_a, epochs=epochs)

Epoch 1/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.0911 - accuracy: 0.9725 - val_loss: 0.0436 - val_accuracy: 0.9802
Epoch 2/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.0174 - accuracy: 0.9956 - val_loss: 0.0229 - val_accuracy: 0.9908
Epoch 3/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.0095 - accuracy: 0.9977 - val_loss: 0.0148 - val_accuracy: 0.9954
Epoch 4/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.0052 - accuracy: 0.9991 - val_loss: 0.0118 - val_accuracy: 0.9963
Epoch 5/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.0032 - accuracy: 0.9995 - val_loss: 0.0078 - val_accuracy: 0.9981
Epoch 6/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.0020 - accuracy: 0.9996 - val_loss: 0.0044 - val_accuracy: 0.9991
Epoch 7/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.0012 - accuracy: 0.9998 - val_loss: 0.0024 - val_accuracy:

In [39]:
history_b = _rank_b_model.fit(train_dataset_b, validation_data=validation_dataset_b, epochs=epochs)

Epoch 1/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.2365 - accuracy: 0.9027 - val_loss: 0.2058 - val_accuracy: 0.9039
Epoch 2/20
1370/1370 [==============================] - 3s 2ms/step - loss: 0.1098 - accuracy: 0.9632 - val_loss: 0.1007 - val_accuracy: 0.9569
Epoch 3/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.0650 - accuracy: 0.9800 - val_loss: 0.0640 - val_accuracy: 0.9760
Epoch 4/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.0479 - accuracy: 0.9857 - val_loss: 0.0520 - val_accuracy: 0.9806
Epoch 5/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.0369 - accuracy: 0.9884 - val_loss: 0.0400 - val_accuracy: 0.9867
Epoch 6/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.0311 - accuracy: 0.9903 - val_loss: 0.0310 - val_accuracy: 0.9903
Epoch 7/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.0257 - accuracy: 0.9922 - val_loss: 0.0249 - val_accuracy:

In [41]:
history_c = _rank_c_model.fit(train_dataset_c, validation_data=validation_dataset_c, epochs=epochs)

Epoch 1/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.2910 - accuracy: 0.8838 - val_loss: 0.4897 - val_accuracy: 0.8053
Epoch 2/20
1370/1370 [==============================] - 3s 3ms/step - loss: 0.1747 - accuracy: 0.9335 - val_loss: 0.3304 - val_accuracy: 0.8438
Epoch 3/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.1106 - accuracy: 0.9644 - val_loss: 0.2610 - val_accuracy: 0.8780
Epoch 4/20
1370/1370 [==============================] - 3s 2ms/step - loss: 0.0806 - accuracy: 0.9760 - val_loss: 0.2044 - val_accuracy: 0.9023
Epoch 5/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.0638 - accuracy: 0.9817 - val_loss: 0.1736 - val_accuracy: 0.9170
Epoch 6/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.0536 - accuracy: 0.9841 - val_loss: 0.1526 - val_accuracy: 0.9296
Epoch 7/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.0451 - accuracy: 0.9867 - val_loss: 0.1342 - val_accuracy:

In [43]:
history_d = _rank_d_model.fit(train_dataset_d, validation_data=validation_dataset_d, epochs=epochs)

Epoch 1/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.3386 - accuracy: 0.8442 - val_loss: 0.4341 - val_accuracy: 0.7829
Epoch 2/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.2269 - accuracy: 0.9172 - val_loss: 0.3482 - val_accuracy: 0.8252
Epoch 3/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.1707 - accuracy: 0.9442 - val_loss: 0.2958 - val_accuracy: 0.8598
Epoch 4/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.1437 - accuracy: 0.9533 - val_loss: 0.2577 - val_accuracy: 0.8807
Epoch 5/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.1254 - accuracy: 0.9599 - val_loss: 0.2309 - val_accuracy: 0.8958
Epoch 6/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.1116 - accuracy: 0.9648 - val_loss: 0.2072 - val_accuracy: 0.9101
Epoch 7/20
1370/1370 [==============================] - 6s 4ms/step - loss: 0.1021 - accuracy: 0.9673 - val_loss: 0.1918 - val_accuracy:

In [45]:
history_bar = _rank_bar_model.fit(train_dataset_bar, validation_data=validation_dataset_bar, epochs=epochs)

Epoch 1/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.1849 - accuracy: 0.9299 - val_loss: 0.3732 - val_accuracy: 0.8916
Epoch 2/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.0972 - accuracy: 0.9625 - val_loss: 0.2943 - val_accuracy: 0.9036
Epoch 3/20
1370/1370 [==============================] - 5s 3ms/step - loss: 0.0829 - accuracy: 0.9684 - val_loss: 0.2370 - val_accuracy: 0.9116
Epoch 4/20
1370/1370 [==============================] - 5s 4ms/step - loss: 0.0708 - accuracy: 0.9733 - val_loss: 0.2251 - val_accuracy: 0.9152
Epoch 5/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.0623 - accuracy: 0.9764 - val_loss: 0.1904 - val_accuracy: 0.9226
Epoch 6/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.0561 - accuracy: 0.9794 - val_loss: 0.1866 - val_accuracy: 0.9243
Epoch 7/20
1370/1370 [==============================] - 4s 3ms/step - loss: 0.0511 - accuracy: 0.9812 - val_loss: 0.1666 - val_accuracy:

In [82]:
_rank_a_model.save('C:/Work/PythonShizai/07_src/C7013/data/rank_model/a/')
_rank_b_model.save('C:/Work/PythonShizai/07_src/C7013/data/rank_model/b/')
_rank_c_model.save('C:/Work/PythonShizai/07_src/C7013/data/rank_model/c/')
_rank_d_model.save('C:/Work/PythonShizai/07_src/C7013/data/rank_model/d/')
_rank_bar_model.save('C:/Work/PythonShizai/07_src/C7013/data/rank_model/bar/')